# Haystack Framework Introduction

This notebook provides a comprehensive introduction to Haystack, demonstrating:
1. **Basic Haystack Pipeline**: Simple question-answering with minimal code
2. **MLflow Integration**: Experiment tracking and model management
3. **SageMaker Deployment**: Production-ready pipeline deployment

In [2]:
%pip install -r requirements.txt

  Using cached haystack_ai-2.14.3-py3-none-any.whl.metadata (14 kB)
  Using cached amazon_bedrock_haystack-3.7.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached more_itertools-10.8.0-py3-none-any.whl.metadata (39 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
Using cached haystack_ai-2.14.3-py3-none-any.whl (515 kB)
Using cached amazon_bedrock_haystack-3.7.0-py3-none-any.whl (38 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.2 MB/s  0:00:00
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
Using cached more_itertools-10.8.0-py3-none-any.whl (69 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [amazon-bedrock-haystack]ck-ai]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
## Setup & Dependencies

import boto3
import mlflow
import sagemaker
import logging
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.generators.amazon_bedrock import AmazonBedrockChatGenerator
from sagemaker.workflow.function_step import step
from sagemaker.workflow.pipeline import Pipeline as SageMakerPipeline
from sagemaker.workflow.parameters import ParameterString
import json
import time
from random import randint

bedrock_model="amazon.nova-lite-v1:0"
# This assumes you have MLFlow set up in SageMaker AI - copy/paste that server ARN here
tracking_server_arn="INSERT_MLFLOW_TRACKING_SERVER_ARN"
experiment_name=f"haystack-demo-{randint(1000,100000)}"

## Part 1: Basic Haystack Demo

Build a simple Haystack Q&A pipeline.

In [4]:
def qna_pipeline():
    pipeline = Pipeline()

    # Prompt Builder - formats the user question
    pipeline.add_component("prompt_builder", ChatPromptBuilder(
        template=[ChatMessage.from_user("Answer this question concisely: {{question}}")],
        required_variables=["question"]
    ))
    
    # LLM Generator - processes the prompt
    pipeline.add_component("llm", AmazonBedrockChatGenerator(model=bedrock_model))
    
    # Connect the components
    pipeline.connect("prompt_builder", "llm")

    return pipeline

In [5]:
# Test the pipeline
test_pipeline = qna_pipeline()
result = test_pipeline.run({"prompt_builder": {"question": "Who was the first president of the United States?"}})

print(result['llm']['replies'][0].text)

INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm


The first president of the United States was George Washington.


## Part 2: MLflow Integration

Now let's add experiment tracking to monitor our pipeline's performance and parameters.

In [48]:
def tracked_qna_pipeline(question: str, tracking_arn: str, experiment_name: str, model_id: str):
    mlflow.set_tracking_uri(tracking_arn)
    #if experiment_name:
        #print(f'experiment name is {experiment_name}; setting in mlflow.')
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run() as run:
        # Log base parameters
        params = {"model": model_id, "question": question}
        mlflow.log_params(params)

        # Run pipeline
        pipeline = qna_pipeline()
        result = pipeline.run({"prompt_builder": {"question": question}})

        # Log results
        answer = result['llm']['replies'][0].text
        mlflow.log_metrics({"answer_length_chars": len(answer)})
        mlflow.log_text(answer, "answer.txt")

        print(f"🔬 Experiment logged with run_id: {run.info.run_id}")
        return answer, run.info.run_id

In [49]:
# Test the tracked pipeline
question = "Explain the benefits of ML orchestration frameworks in simple terms."
answer, run_id = tracked_qna_pipeline(question, experiment_name=experiment_name, tracking_arn=tracking_server_arn, model_id=bedrock_model)
print(f"Q: {question}")
print(f"A: {answer[:100]}...")

INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm


🔬 Experiment logged with run_id: 6be884055e43430698f49a5da8a84882
🏃 View run stately-mole-102 at: https://us-east-1.experiments.sagemaker.aws/#/experiments/2/runs/6be884055e43430698f49a5da8a84882
🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/2
Q: Explain the benefits of ML orchestration frameworks in simple terms.
A: ML orchestration frameworks help manage and automate the complex tasks involved in machine learning ...


## Part 3: SageMaker Pipeline Deployment

Finally, let's deploy our Haystack pipeline to SageMaker for production use.

In [58]:
# Create SageMaker Pipeline object

# Define input parameters
question_param = ParameterString(name="Question")
model_param = ParameterString(name="ModelId")
tracking_server_arn_param = ParameterString(name="TrackingServerArn")
experiment_name_param = ParameterString(name="ExperimentName")

# Create one pipeline step
inference_step = step(
    tracked_qna_pipeline,
    name="haystack-qna-pipeline",
    keep_alive_period_in_seconds=300,
    dependencies="requirements-sagemaker.txt"
)(
    question=question_param,
    model_id=model_param,
    tracking_arn=tracking_server_arn_param,
    experiment_name=experiment_name_param
)

# Create the pipeline
sm_pipeline = SageMakerPipeline(
    name="haystack-intro-pipeline",
    parameters=[question_param, model_param, tracking_server_arn_param, experiment_name_param],
    steps=[inference_step],
    sagemaker_session=sagemaker.Session()
)

In [59]:
# Deploy and run the the pipeline
# This requires IAM permissions, and assumes we are running from a SageMaker Studio notebook

# Set desired logging level
for logger_name in ['sagemaker', 'sagemaker.remote_function', 'sagemaker.workflow', 'sagemaker.config']:
    logging.getLogger(logger_name).setLevel(logging.ERROR)

# deploy the pipeline
sm_pipeline.upsert(role_arn=sagemaker.get_execution_role())

execution = sm_pipeline.start(
    parameters={
        "Question": "How does the Haystack framework work?",
        "ModelId": bedrock_model,
        "TrackingServerArn": tracking_server_arn,
        "ExperimentName": experiment_name
    }
)
print(f"📊 Pipeline execution started: {execution.arn}")

📊 Pipeline execution started: arn:aws:sagemaker:us-east-1:933591302813:pipeline/haystack-intro-pipeline/execution/enomc0s0ewre


## Summary

This notebook demonstrated three key aspects of building production LLM applications:
- The use of Haystack to build a Q&A pipeline with just 3 components.
- Automate experiment logging, with parameter and metric tracking, in MLFlow.
- Deployment to SageMaker Pipelines for production-ready pipeline orchestration on scalable, managed infrastructre.

For more complex examples, see:
- `local_pipe/pipe_v2.ipynb` - Advanced RAG with tool routing
- `sagemaker/main.ipynb` - Full MLOps pipeline with quality gates